<a href="https://colab.research.google.com/github/arteagac/xlogit/blob/master/examples/mixed_logit_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mixed Logit

The purpose of this notebook is to provide users with a step-by-step guide for estimating mixed logit models using xlogit package. 

## Install and import `xlogit` package

Install `xlogit` package using pip install as shown below:

In [1]:
!pip install git+https://github.com/arteagac/xlogit
from xlogit import MixedLogit
MixedLogit.check_if_gpu_available()

  Cloning https://github.com/arteagac/xlogit to /tmp/pip-req-build-2knpiu0c
  Running command git clone -q https://github.com/arteagac/xlogit /tmp/pip-req-build-2knpiu0c
  Created wheel for xlogit: filename=xlogit-0.1.0-cp36-none-any.whl size=16129 sha256=7f5d058beade3f9a809f1683946a89987818cab966c08d6bc0643d6f39e1f055
  Stored in directory: /tmp/pip-ephem-wheel-cache-74f3htzb/wheels/64/50/8d/a97e0500aac20b521a2896234d6598045323a7d0daca37648a
Successfully built xlogit
1 GPU device(s) available. xlogit will use GPU processing


True

## Electricity Dataset

For the first example, we use the Electricity dataset from the study https://escholarship.org/content/qt1900p96t/qt1900p96t.pdf. This dataset is popularly used in examples of R's mlogit package and can be downloaded from https://raw.githubusercontent.com/arteagac/xlogit/master/examples/data/electricity_long.csv" in the long format. The dataset is  from a stated choice experiment conducted to analyse customers' preferences towards four hypothetical electricity suppliers. 

### Read data

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("https://raw.githubusercontent.com/arteagac/xlogit/master/examples/data/electricity_long.csv")
df

,choice,id,alt,pf,cl,loc,wk,tod,seas,chid
0,0,1,1,7,5,0,1,0,0,1
1,0,1,2,9,1,1,0,0,0,1
2,0,1,3,0,0,0,0,0,1,1
3,1,1,4,0,5,0,1,1,0,1
4,0,1,1,7,0,0,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...
17227,0,361,4,0,1,1,0,0,1,4307
17228,1,361,1,9,0,0,1,0,0,4308
17229,0,361,2,7,0,0,0,0,0,4308
17230,0,361,3,0,1,0,1,0,1,4308


The dataset is in panel form, with each individual reporting preferences for upto 12 choice situations. Since, all inidividuals have not responded to all the 12 situations, the dataset in an unbalanced panel. 361 individuals were interviewed with a total of 4,308 observations. See https://cran.r-project.org/web/packages/mlogit/vignettes/e3mxlogit.html for more details on the attributes and the choice analyses.

### Fit the model

The data needs to be in long format. The user inputs required to fit the model are as follows:

1.   `X`: dataframe columns with respect to varnames
2.   `y`: dataframe column containing the choice outcome
3.   `varnames`: list containing all the explanatory variable names to be included in the model 
4.   `isvars`: list of individual-specific variables in varnames
5.   `alts`: dataframe column containing the alternative ids
6.   `randvars`: dictionary of mixing distributions. Possible distributions include 'n'-normal; 'u'-uniform; 'ln'-lognormal; 'tn'-truncated normal; 't'-triangular
7.   `panels`: dataframe column containing the unique individual id
8.   `n_draws`: number of random draws for the cofficients (default value is 100)

The model.fit object from class MixedLogit is called to fit the model. The fit results can be seen using model.summary().

In [ ]:
varnames = ["pf", "cl", "loc", "wk", "tod", "seas"]

X = df[varnames].values
y = df['choice'].values


from xlogit import MixedLogit
model = MixedLogit()
model.fit(X, y, 
          varnames, 
          alts=df['alt'], 
          randvars={'pf': 'n','cl':'n','loc':'n','wk':'n','tod':'n','seas':'n'}, 
          panels=df.id.values,
          n_draws=600)
model.summary()

Estimation with GPU processing enabled.
Optimization terminated successfully.
         Current function value: 3888.413414
         Iterations: 46
         Function evaluations: 51
         Gradient evaluations: 51
Estimation time= 5.2 seconds
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
pf                     -0.9996286     0.0331488   -30.1557541     9.98e-100 ***
cl                     -0.2355334     0.0220401   -10.6865870      1.97e-22 ***
loc                     2.2307891     0.1164263    19.1605300      5.64e-56 ***
wk                      1.6251657     0.0918755    17.6887855      6.85e-50 ***
tod                    -9.6067367     0.3112721   -30.8628296     2.36e-102 ***
seas                   -9.7892800     0.2913063   -33.6047603     2.81e-112 ***
sd.pf                   0.2357813     0.0181892 

The xlogit estimates are similar to those estimated using R's mlogit package (https://cran.r-project.org/web/packages/mlogit/vignettes/e3mxlogit.html). With GPU enables estimations, xlogit estimates the model in less than 20 seconds, significantly faster than open-source pacakges such as mlogit and Biogeme. This feature can be beneficial while fitting models for large datasets with multiple explanatory variables to be estimated with random coefficients.

Other random distributions for coefficients can also used. We will now use logarithmic distributions of 'tod' and 'seas' variables. Since these are price related variables, we expect to obtain negative coefficients. Therefore, we will first reverse their signs in the dataframe as shown:

In [ ]:
df['tod'] = -df['tod']
df['seas'] = -df['seas']

In [ ]:
varnames = ["pf", "cl", "loc", "wk", "tod", "seas"]

X = df[varnames].values
y = df['choice'].values


from xlogit import MixedLogit
model = MixedLogit()
model.fit(X, y, 
          varnames, 
          alts=df['alt'], 
          randvars={'pf': 'n','cl':'n','loc':'n','wk':'n','tod':'ln','seas':'ln'}, 
          panels=df.id.values,
          n_draws=600)
model.summary()

Estimation with GPU processing enabled.
Optimization terminated successfully.
         Current function value: 3889.280108
         Iterations: 36
         Function evaluations: 46
         Gradient evaluations: 46
Estimation time= 4.5 seconds
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
pf                     -0.9824654     0.0378644   -25.9469741      1.46e-83 ***
cl                     -0.2246626     0.0239780    -9.3695389      6.18e-18 ***
loc                     2.2789672     0.1147575    19.8589830      7.29e-59 ***
wk                      1.6069950     0.0885660    18.1446096      8.97e-52 ***
tod                     2.2584029     0.0364832    61.9025907     1.36e-193 ***
seas                    2.2559813     0.0332887    67.7701438     1.06e-206 ***
sd.pf                   0.2181707     0.0191448 

## Fishing Dataset

The second example uses the revealed preferences dataset of fishing mode choice of 1,182 individuals. The dataset is also open-source dataset and is used in mlogit examples. It can be downloaded from https://raw.githubusercontent.com/arteagac/xlogit/master/examples/data/fishing_long.csv in long format. More information on the dataset can be found in http://www2.uaem.mx/r-mirror/web/packages/mlogit/vignettes/mlogit.pdf

### Read data

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/arteagac/xlogit/master/examples/data/fishing_long.csv")
df

,id,alt,choice,income,price,catch
0,1,beach,0,7083.33170,157.930,0.0678
1,1,boat,0,7083.33170,157.930,0.2601
2,1,charter,1,7083.33170,182.930,0.5391
3,1,pier,0,7083.33170,157.930,0.0503
4,2,beach,0,1249.99980,15.114,0.1049
...,...,...,...,...,...,...
4723,1181,pier,0,416.66668,36.636,0.4522
4724,1182,beach,0,6250.00130,339.890,0.2537
4725,1182,boat,1,6250.00130,235.436,0.6817
4726,1182,charter,0,6250.00130,260.436,2.3014


Four alternatives are considered in the dataset: beach, boat, charter and pier. There are two alternative-specific variables: 'price' and 'catch' and one individual-specific variable 'income'. We can estimate alternative-specific intercepts by using the fit_intercept option. First alternative is the default whose coefficeint value is set to zero. You can change the default alternative using the base_alt option.

### Fit model

In [ ]:
varnames = ['price','catch']
X = df[varnames].values
y = df['choice'].values

from xlogit import MixedLogit
model = MixedLogit()
model.fit(X, y, varnames= varnames,
          alts=['beach', 'boat', 'charter', 'pier'],
          randvars = {'price': 'n', 'catch': 'n'}, fit_intercept = True)
model.summary()

Estimation with GPU processing enabled.
Optimization terminated successfully.
         Current function value: 1203.083622
         Iterations: 49
         Function evaluations: 64
         Gradient evaluations: 64
Estimation time= 1.1 seconds
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
_intercept.boat         0.8011318     0.1326072     6.0413913      1.23e-08 ***
_intercept.charter      2.0808304     0.1896238    10.9734665      9.56e-26 ***
_intercept.pier         0.3052972     0.1152324     2.6494051         0.024 *  
price                  -0.0487159     0.0055495    -8.7784157      4.76e-17 ***
catch                   0.4422202     0.1882570     2.3490239        0.0507 .  
sd.price                0.0252920     0.0041769     6.0552204      1.13e-08 ***
sd.catch               -1.3893436     0.6015510 

## SwissMetro Dataset

### Read data

In [3]:
import pandas as pd
import numpy as np

df_wide = pd.read_table("http://transp-or.epfl.ch/data/swissmetro.dat", sep="\t")

# Keep only samples with known choice and purpose commute or business
df_wide = df_wide[(df_wide['PURPOSE'].isin([1, 3]) & (df_wide['CHOICE'] != 0))]
df_wide['custom_id'] = np.arange(len(df_wide))  # Add unique identifier column

df_wide

,GROUP,SURVEY,SP,ID,PURPOSE,FIRST,TICKET,WHO,LUGGAGE,AGE,MALE,INCOME,GA,ORIGIN,DEST,TRAIN_AV,CAR_AV,SM_AV,TRAIN_TT,TRAIN_CO,TRAIN_HE,SM_TT,SM_CO,SM_HE,SM_SEATS,CAR_TT,CAR_CO,CHOICE,custom_id
0,2,0,1,1,1,0,1,1,0,3,0,2,0,2,1,1,1,1,112,48,120,63,52,20,0,117,65,2,0
1,2,0,1,1,1,0,1,1,0,3,0,2,0,2,1,1,1,1,103,48,30,60,49,10,0,117,84,2,1
2,2,0,1,1,1,0,1,1,0,3,0,2,0,2,1,1,1,1,130,48,60,67,58,30,0,117,52,2,2
3,2,0,1,1,1,0,1,1,0,3,0,2,0,2,1,1,1,1,103,40,30,63,52,20,0,72,52,2,3
4,2,0,1,1,1,0,1,1,0,3,0,2,0,2,1,1,1,1,130,36,60,63,42,20,0,90,84,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8446,3,1,1,939,3,1,7,3,1,5,1,2,0,1,2,1,1,1,108,13,30,50,17,30,0,130,64,1,6763
8447,3,1,1,939,3,1,7,3,1,5,1,2,0,1,2,1,1,1,108,12,30,53,16,10,0,80,80,1,6764
8448,3,1,1,939,3,1,7,3,1,5,1,2,0,1,2,1,1,1,108,16,60,50,16,20,0,80,64,1,6765
8449,3,1,1,939,3,1,7,3,1,5,1,2,0,1,2,1,1,1,128,16,30,53,17,30,0,80,104,1,6766


### Convert data to long format

In [4]:
# Rename columns to match format expected by pandas (first varname and then suffix)
df_wide.rename(columns={"TRAIN_TT": "time_train", "SM_TT": "time_sm", "CAR_TT": "time_car",
                        "TRAIN_CO": "cost_train","SM_CO": "cost_sm", "CAR_CO": "cost_car",
                        "TRAIN_HE": "headway_train", "SM_HE": "headway_sm", "SM_SEATS": "seatconf_sm",
                        "TRAIN_AV": "av_train", "SM_AV": "av_sm","CAR_AV": "av_car"}, inplace=True)
 
# Convert from wide to long format using pandas.
df = pd.wide_to_long(df_wide, ["time", "cost", "headway", "seatconf", "av"],
                     i="custom_id", j="alt", sep="_", suffix='\w+').sort_values(
                         by=['custom_id', 'alt']).reset_index()

# Fill unexisting values for some alternatives
df = df.fillna(0)  
# Format the outcome variable approapriatly
df["CHOICE"] = df["CHOICE"].map({1: 'train', 2:'sm', 3: 'car'})
# Convert CHOICE to True if alternative was selected; False otherwise
df["CHOICE"] = df["CHOICE"] == df["alt"]
df

,custom_id,alt,ORIGIN,PURPOSE,MALE,TICKET,SP,DEST,ID,WHO,SURVEY,GA,CHOICE,INCOME,GROUP,FIRST,LUGGAGE,AGE,time,cost,headway,seatconf,av
0,0,car,2,1,0,1,1,1,1,1,0,0,False,2,2,0,0,3,117,65,0.0,0.0,1
1,0,sm,2,1,0,1,1,1,1,1,0,0,True,2,2,0,0,3,63,52,20.0,0.0,1
2,0,train,2,1,0,1,1,1,1,1,0,0,False,2,2,0,0,3,112,48,120.0,0.0,1
3,1,car,2,1,0,1,1,1,1,1,0,0,False,2,2,0,0,3,117,84,0.0,0.0,1
4,1,sm,2,1,0,1,1,1,1,1,0,0,True,2,2,0,0,3,60,49,10.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20299,6766,sm,1,3,1,7,1,2,939,3,1,0,False,2,3,1,1,5,53,17,30.0,0.0,1
20300,6766,train,1,3,1,7,1,2,939,3,1,0,True,2,3,1,1,5,128,16,30.0,0.0,1
20301,6767,car,1,3,1,7,1,2,939,3,1,0,False,2,3,1,1,5,100,80,0.0,0.0,1
20302,6767,sm,1,3,1,7,1,2,939,3,1,0,False,2,3,1,1,5,53,21,30.0,0.0,1


### Create specification

In [5]:
# Scale variables
df['time'] = df['time']/100
train_pass = ((df["GA"] == 1) & (df["alt"].isin(['train', 'sm']))).astype(int)
df['cost'] = df['cost']*(train_pass==0)/100
 
# Create alternative specific constants
df['asc_train'] = np.ones(len(df))*(df['alt'] == 'train')
df['asc_car'] = np.ones(len(df))*(df['alt'] == 'car')

### Estimate the model

In [6]:
from xlogit import MixedLogit

varnames=['asc_car', 'asc_train', 'cost', 'time']
model = MixedLogit()
model.fit(X=df[varnames], y=df['CHOICE'], varnames=varnames, alts=df['alt'],
          ids=df['custom_id'], avail=df['av'], randvars={'time': 'n'}, n_draws=2000)
model.summary()

Estimation with GPU processing enabled.
Optimization terminated successfully.
         Current function value: 5214.927430
         Iterations: 19
         Function evaluations: 25
         Gradient evaluations: 25
Estimation time= 6.7 seconds
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
asc_car                 0.1371110     0.0489429     2.8014468        0.0158 *  
asc_train              -0.4018231     0.0586071    -6.8562141      5.35e-11 ***
cost                   -1.2854009     0.0614645   -20.9128918      7.27e-93 ***
time                   -2.2598763     0.1111418   -20.3332649      5.58e-88 ***
sd.time                 1.6576641     0.1308694    12.6665510      2.91e-35 ***
---------------------------------------------------------------------------
Significance:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 

The estimates are very similar to the ones returned by biogeme ([see biogeme results here](https://biogeme.epfl.ch/examples/swissmetro/05normalMixture.html)). The slight differences are due to the different random draws used. Note that this estimation took around 6 seconds using 2,000 random draws. 

## Car Dataset

The third example uses a stated preference panel dataset for choice of car. Three alternatives are considered, with upto 6 choice situations per individual. This again is an unbalanced panel with responses of some individuals less than 6 situations. The dataset contains 8 explanaotry variables: price, operating cost, range, and binary indicators to indicate whether the car is electric, hybrid, and if performance is high or medium respectively. The dataset can be downloaded from https://raw.githubusercontent.com/arteagac/xlogit/master/examples/data/car100_long.csv in the long format.

### Read data

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("https://raw.githubusercontent.com/arteagac/xlogit/master/examples/data/car100_long.csv")


Since price and operating cost need to be estimated with negative coefficients, we reverse the variable signs in the dataframe. 

In [ ]:
df.price = -1*df.price/10000
df.operating_cost = -1*df.operating_cost
df

,person_id,choice_situation,choice,price,operating_cost,range,electric,gas,hybrid,high_performance,medium_performance
0,1,1,0,-4.6763,-47.43,0.0,0,0,1,0,0
1,1,1,1,-5.7209,-27.43,1.3,1,0,0,1,1
2,1,1,0,-8.7960,-32.41,1.2,1,0,0,0,1
3,1,2,1,-3.3768,-4.89,1.3,1,0,0,1,1
4,1,2,0,-9.0336,-30.19,0.0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
4447,100,1483,0,-2.8036,-14.45,1.6,1,0,0,0,0
4448,100,1483,0,-1.9360,-54.76,0.0,0,1,0,1,1
4449,100,1484,1,-2.4054,-50.57,0.0,0,1,0,0,0
4450,100,1484,0,-5.2795,-21.25,0.0,0,0,1,0,1


### Fit the model

In [ ]:
varnames = ['high_performance','medium_performance','price', 'operating_cost',
            'range', 'electric', 'hybrid'] 

X = df[varnames].values
y = df['choice'].values

from xlogit import MixedLogit
model = MixedLogit()
model.fit(X, y, varnames = varnames,
          alts=['car','bus','bike'],
          randvars = {'price': 'ln', 'operating_cost': 'n',
                      'range': 'ln', 'electric':'n', 'hybrid': 'n'}, 
          panels=df.person_id.values, #Panel column
          n_draws = 100) 
model.summary()

Estimation with GPU processing enabled.
Optimization terminated successfully.
         Current function value: 1297.937510
         Iterations: 52
         Function evaluations: 63
         Gradient evaluations: 63
Estimation time= 1.0 seconds
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
high_performance        0.1059300     0.0923764     1.1467216         0.411    
medium_performance      0.5660352     0.0961273     5.8883953      2.36e-07 ***
price                  -0.7861318     0.1358040    -5.7887229      3.65e-07 ***
operating_cost          0.0120780     0.0057487     2.1009905        0.0898 .  
range                  -0.5886938     0.3564083    -1.6517401         0.204    
electric               -1.6330363     0.3232833    -5.0514085      8.25e-06 ***
hybrid                  0.6902022     0.1474823 